In [1]:
%reset
import gc
# tensorflow.keras.backend.clear_session()
gc.collect()

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


0

In [1]:
import os
import random
import numpy as np
import pandas as pd 
# from skimage import io
# from skimage import color
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt
# import seaborn as sns
# from dask.array.image import imread
# from dask import bag, threaded
# from dask.diagnostics import ProgressBar
import cv2
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")



import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import optimizers
#Use the generated model 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense


In [10]:
driver_details = pd.read_csv('../../statefarm_dataset/driver_imgs_list.csv',na_values='na')
print(driver_details.head(5))


  subject classname            img
0    p002        c0  img_44733.jpg
1    p002        c0  img_72999.jpg
2    p002        c0  img_25094.jpg
3    p002        c0  img_69092.jpg
4    p002        c0  img_92629.jpg


In [11]:
## Getting all the images

train_image = []
image_label = []


for i in range(10):
    print('now we are in the folder C',i)
    imgs = os.listdir("../../statefarm_dataset/train/c"+str(i))
    print(len(imgs))
    for j in range(len(imgs)):   
        img_name = "../../statefarm_dataset/train/c"+str(i)+"/"+imgs[j]
        img = cv2.imread(img_name)
        img = img[50:,120:-50]
        img = cv2.resize(img,(224,224))
        label = i
        driver = driver_details[driver_details['img'] == imgs[j]]['subject'].values[0]
        train_image.append([img,label,driver])
        image_label.append(i)



now we are in the folder C 0
2489
now we are in the folder C 1
2267
now we are in the folder C 2
2317
now we are in the folder C 3
2346
now we are in the folder C 4
2326
now we are in the folder C 5
2312
now we are in the folder C 6
2325
now we are in the folder C 7
2002
now we are in the folder C 8
1911
now we are in the folder C 9
2129


In [ ]:
## getting list of driver names

D = []
for features,labels,drivers in train_image:
    D.append(drivers)

## Deduplicating drivers

deduped = []

for i in D:
    if i not in deduped:
        deduped.append(i)
    

## selecting random drivers for the validation set
driv_selected = []
import random
driv_nums = random.sample(range(len(deduped)), 4)
for i in driv_nums:
    driv_selected.append(deduped[i])
print(driv_selected)

In [12]:
## Randomly shuffling the images

import random
random.shuffle(train_image)

driv_selected = ['p050', 'p015', 'p022', 'p056']

In [13]:
## Splitting the train and test

X_train= []
y_train = []
X_test = []
y_test = []
D_train = []
D_test = []

for features,labels,drivers in train_image:
    if drivers in driv_selected:
        X_test.append(features)
        y_test.append(labels)
        D_test.append(drivers)
    
    else:
        X_train.append(features)
        y_train.append(labels)
        D_train.append(drivers)
    
print (len(X_train),len(X_test))
print (len(y_train),len(y_test))

## Converting images to nparray. Encoding the Y

X_train = np.array(X_train).reshape(-1,224,224,3)
X_test = np.array(X_test).reshape(-1,224,224,3)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


print (X_train.shape)


18732 3692
18732 3692
(18732, 224, 224, 3)


In [ ]:
# from tensorflow.keras.models import model_from_json
# with open('mobilenet_sgd_nolayers.json','r') as f:
#     model = model_from_json(f.read())
# model.summary()

In [14]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet import preprocess_input

base_model=MobileNet(include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.
# base_model.summary()

x=base_model.output
x=GlobalAveragePooling2D()(x)

preds=Dense(10,activation='softmax')(x) #final layer with softmax activation

model = Model(inputs=base_model.input, outputs=preds)
sgd = optimizers.SGD(lr = 0.005) # try 0.01 - didn't converge and 0.005 , 0.001 best acc of 11%

model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128 

In [ ]:
# Random Eraser
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

def blur(img):
    return (cv2.blur(img,(5,5)))

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
import json
import datetime

checkpointer = ModelCheckpoint('models/mobilenetv1_28mar.hdf5', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

log_dir="logs_mobilenetv1_28mar/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


datagen = ImageDataGenerator(
                               height_shift_range=0.5,
                               width_shift_range=0.5,
                               shear_range=0.2,
                               rotation_range=30,
                               zoom_range=0.5,
                               horizontal_flip=True,
                               preprocessing_function=get_random_eraser(v_l=0, v_h=1)
        )

data_generator = datagen.flow(X_train, y_train, batch_size = 64)

# Fits the model on batches with real-time data augmentation:
mobilenet_model = model.fit_generator(data_generator,steps_per_epoch = len(X_train) / 64, 
                                      callbacks=[checkpointer, earlystopper, tensorboard_callback],
                                      epochs = 20, verbose = 1, validation_data = (X_test, y_test))

# model_json = model.to_json()
# with open("mobilenetv1.json",'w') as json_file:
#  	json_file.write(model_json)

model.save_weights("models/mobilenetv1_28mar.h5")
print("Model saved")


Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 292.6875 steps, validate on 3692 samples
Epoch 1/20
292/292 [============================>.] - ETA: 0s - loss: 1.3803 - accuracy: 0.5593
Epoch 00001: val_loss improved from inf to 0.93621, saving model to models/mobilenetv1_28mar.hdf5
293/292 [==============================] - 421s 1s/step - loss: 1.3778 - accuracy: 0.5603 - val_loss: 0.9362 - val_accuracy: 0.6937
Epoch 2/20
292/292 [============================>.] - ETA: 0s - loss: 0.5299 - accuracy: 0.8582
Epoch 00002: val_loss improved from 0.93621 to 0.67118, saving model to models/mobilenetv1_28mar.hdf5
293/292 [==============================] - 157s 536ms/step - loss: 0.5297 - accuracy: 0.8583 - val_loss: 0.6712 - val_accuracy: 0.7757
Epoch 3/20
292/292 [============================>.] - ETA: 0s - loss: 0.3511 - accuracy: 0.9051
Epoch 00003: val_loss improved from 0.67118 to 0.59716, saving model to models/mobilenetv1_28

In [1]:

tags = { "C0": "safe driving",
"C1": "texting - right",
"C2": "talking on the phone - right",
"C3": "texting - left",
"C4": "talking on the phone - left",
"C5": "operating the radio",
"C6": "drinking",
"C7": "reaching behind",
"C8": "hair and makeup",
"C9": "talking to passenger" }

from keras.models import model_from_json,Model
from keras.applications import MobileNetV3Large
from keras.layers import GlobalAveragePooling2D, Dense
from keras import optimizers

base_model=MobileNetV3Large(alpha=1.0,weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer..4

x=base_model.output
x=GlobalAveragePooling2D()(x)

preds=Dense(8,activation='softmax')(x) #final layer with softmax activation
model = Model(inputs=base_model.input, outputs=preds)




# with open('models/mobilenetv3_large_json.json','r') as f:
#     model = model_from_json(f.read())

# model.load_weights('models/mobilenetv3_large.hdf5')

sgd = optimizers.SGD(lr = 0.005) # try 0.01 - didn't converge and 0.005 , 0.001 best acc of 11%

model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

model_json = model.to_json()

with open("mobilenetv3_large.json",'w') as json_file:
 	json_file.write(model_json)







Using TensorFlow backend.
/home/mcw/anaconda3/envs/subhadms/lib/python3.7/site-packages/keras/applications/mobilenet_v3.py:347: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
Conv_pad (ZeroPadding2D)        (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv (Conv2D)                   (None, None, None, 1 432         Conv_pad[0][0]                   
__________________________________________________________________________________________________
Conv/BatchNorm (BatchNormalizat (None, None, None, 1 64          Conv[0][0]                       
____________________________________________________________________________________________

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
# labels is the image array
test_image = []
i = 0
fig, ax = plt.subplots(1, 20, figsize = (50,50 ))

files = os.listdir('../../statefarm_dataset/test')
nums = np.random.randint(low=1, high=len(files), size=20)
for i in range(20):
#     print ('Image number:',i)
    img = cv2.imread('../../statefarm_dataset/test/'+files[nums[i]])
    #img = color.rgb2gray(img)
    img = img[50:,120:-50]
    img = cv2.resize(img,(224,224))
    test_image.append(img)
    ax[i].imshow(img,cmap = 'gray')
    plt.show

test = []

for img in test_image:
    test.append(img)


In [ ]:
predict_test = np.array(test).reshape(-1,224,224,3).astype('float32')
prediction = model.predict(predict_test)
print(prediction[0])


# labels is the image array
i = 0
fig, ax = plt.subplots(20, 1, figsize = (100,100))

for i in range(20):
    ax[i].imshow(test[i])
    predicted_class = 'C'+str(np.where(prediction[i] == np.amax(prediction[i]))[0][0])
    ax[i].set_title(tags[predicted_class])
    plt.show
    


In [ ]:
!pip install natsort